step 2 : translate.ipynb (NLLB 번역 파트)

In [ ]:
from transformers import pipeline

# 1. 번역 파이프라인 준비
translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")



Device set to use cpu


In [ ]:

# 2. 원본 텍스트 로드
with open("data/segments/AAPL_2023_Q3_joined.txt", "r", encoding="utf-8") as f:
    original_text = f.read()

# 3. 특수 구분자 보호 (␟ → __SEP__)
#    \n\n␟\n\n 형태로 정리된 경우 줄바꿈도 함께 유지
protected_text = original_text.replace("\n\n␟\n\n", "\n\n__SEP__\n\n")



In [47]:
protected_text

'aapl-20230701\n\n__SEP__\n\nUNITED STATES\n\n__SEP__\n\nSECURITIES AND EXCHANGE COMMISSION\n\n__SEP__\n\nWashington, D.C. 20549\n\n__SEP__\n\nFORM 10-Q\n\n__SEP__\n\n(Mark One)\n\n__SEP__\n\nQUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\n__SEP__\n\nor\n\n__SEP__\n\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\n__SEP__\n\nFor the transition period from\n\n__SEP__\n\nto\n\n__SEP__\n\nCommission File Number:\n\n__SEP__\n\nApple Inc.\n\n__SEP__\n\n(Exact name of Registrant as specified in its charter)\n\n__SEP__\n\nCalifornia\n\n__SEP__\n\n(State or other jurisdiction\n\n__SEP__\n\nof incorporation or organization)\n\n__SEP__\n\n(I.R.S. Employer Identification No.)\n\n__SEP__\n\nOne Apple Park Way\n\n__SEP__\n\nCupertino, California\n\n__SEP__\n\n(Address of principal executive offices)\n\n__SEP__\n\n(Zip Code)\n\n__SEP__\n\nSecurities registered pursuant to Section 12(b) of the Act:\n\n__SEP__\n\nTi

In [48]:
# 4. 한 번에 번역 실행
translated_output = translator(
    protected_text,
    src_lang="en",          # 또는 "eng_Latn"
    tgt_lang="kor_Hang",
    do_sample=False,        # 결정론적
    num_beams=4             # 번역 품질 향상
)[0]['translation_text']


Token indices sequence length is longer than the specified maximum sequence length for this model (8818 > 1024). Running this sequence through the model will result in indexing errors
Your input_length: 8818 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [49]:
translated_output

'__ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __ __'

In [50]:

# 5. 보호했던 구분자 복원 (__SEP__ → ␟)
restored_output = translated_output.replace("__SEP__", "␟")

# 6. 번역된 결과 저장
with open("data/translations/AAPL_2023_Q3_translated.txt", "w", encoding="utf-8") as f:
    f.write(restored_output)

In [43]:
from transformers import pipeline
# 번역 파이프라인 준비
translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")

Device set to use cpu


다시해볼까 다시다시


In [52]:
from transformers import pipeline, AutoTokenizer

# 1. 번역 파이프라인 & 토크나이저 준비
model_name = "facebook/nllb-200-distilled-600M"
translator = pipeline("translation", model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Device set to use cpu


In [53]:


# 2. 원본 텍스트 로드
with open("data/segments/AAPL_2023_Q3_joined.txt", "r", encoding="utf-8") as f:
    original_text = f.read()

# 3. 구분자 보호
protected_text = original_text.replace("\n\n␟\n\n", "\n\n__SEP__\n\n")

# 4. 텍스트를 문장 단위로 분할 (간단하게 \n 기준)
lines = protected_text.split("\n")

# 5. 문장을 모아서 1024 토큰 이하로 나눔
chunks = []
current_chunk = ""
max_token = 1000

for line in lines:
    candidate = current_chunk + "\n" + line if current_chunk else line
    tokenized = tokenizer(candidate, return_tensors="pt", truncation=False)
    if tokenized["input_ids"].shape[1] <= max_token:
        current_chunk = candidate
    else:
        chunks.append(current_chunk)
        current_chunk = line
if current_chunk:
    chunks.append(current_chunk)

print(f"✅ {len(chunks)}개의 청크로 분할됨")

# 6. 각 chunk 번역
translated_chunks = []

for idx, chunk in enumerate(chunks):
    print(f"\n🔄 청크 {idx+1}/{len(chunks)} 번역 중...")
    translated = translator(
        chunk,
        src_lang="en",
        tgt_lang="kor_Hang",
        do_sample=False,
        num_beams=4,
        max_length=1024 
    )[0]['translation_text']
    print(f"📝 번역 결과 일부:\n{translated[:300]}...\n")  # 앞 300자만 출력
    translated_chunks.append(translated)

# 7. 구분자 복원
final_output = "\n".join(translated_chunks).replace("__SEP__", "␟")

# 8. 저장
with open("data/translations/AAPL_2023_Q3_translated.txt", "w", encoding="utf-8") as f:
    f.write(final_output)

print("\n✅ 전체 번역 완료. 저장됨: data/translations/AAPL_2023_Q3_translated.txt")


Token indices sequence length is longer than the specified maximum sequence length for this model (1026 > 1024). Running this sequence through the model will result in indexing errors
Your input_length: 1024 is bigger than 0.9 * max_length: 1024. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


✅ 9개의 청크로 분할됨

🔄 청크 1/9 번역 중...


KeyboardInterrupt: 

In [44]:

# # HTML 파일 불러오기
# # load original segments
# with open("data/segments/AAPL_2023_Q3_joined.txt", "r", encoding="utf-8") as f:
#     segments = f.read().split("\n\n␟\n\n")

# translations = []
# for seg in segments:
#     if seg.strip():
#         result = translator(seg, src_lang="en", tgt_lang="kor_Hang",num_beams=2, penalty_alpha=0.6, top_k=4,)[0]['translation_text']
#     else:
#         result = ""
#     translations.append(result)

# # save to file
# with open("data/translations/AAPL_2023_Q3_translated.txt", "w", encoding="utf-8") as f:
#     f.write("\n\n␟\n\n".join(translations))


In [5]:
# key: value  
# <eng>: korean
translation_cache_table = {}

In [15]:
result = translator("APPL", src_lang="en", tgt_lang="kor_Hang",num_beams=2, penalty_alpha=0.6, top_k=1, top_p=1, do_sample=True)[0]['translation_text']
result

'APPL에서 APPL을 사용합니다.'

In [84]:
result = translator('''
aapl-20230701

1

UNITED STATES

1

SECURITIES AND EXCHANGE COMMISSION

1

Washington, D.C. 20549

1

FORM 10-Q

1

(Mark One)

1

QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

1

or

1

TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

1

For the transition period from

1

to

1

Commission File Number:

1

Apple Inc.

1

(Exact name of Registrant as specified in its charter)

1

California

''', src_lang="en", tgt_lang="kor_Hang", 
num_beams=1,             
penalty_alpha=None)[0]['translation_text']         
result

'aapl-20230701 1 미국 1 증권 및 거래소 위원회 1 워싱턴 DC 20549 1 양식 10-Q 1 (Marken One) 1 방정식 보고서 SECTION 13 또는 15에 관한 법률 1 또는 1 전환 보고서 1 또는 1934 SECURITIES EXCHANGE ACT의 SECTION 13 또는 15에 관한 법률 1 또는 1 전환 보고서 1 미국 증권 거래소 법 1934의 SECURITIES EXCHANGE ACT의 전환 기간 1 1에서 1 파일 번호: 1 애플 Inc. 1 (기본에 명시된 등록자의 정확한 이름) 1 캘리포니아 '

In [42]:
result[:100]

'aapl-20230701  미국 증권 및 거래소위원회  워싱턴 DC 20549  FORM 10-Q  (Mark One)  SECURITIES EXCHANGE ACT OF 1934 '

두번째태그달아서해보기시도


In [77]:
# 1. load and split by ␟
with open("data/segments/AAPL_2023_Q3_joined.txt", "r", encoding="utf-8") as f:
    original = f.read()

segments = original.strip().split("\n\n␟\n\n")  # 구조 보존용
tagged_segments = [f"<SEG{i}>{seg}" for i, seg in enumerate(segments)]
tagged_text = "\n".join(tagged_segments)


In [78]:
from transformers import pipeline

translator = pipeline("translation", model="facebook/nllb-200-distilled-600M")

translated_text = translator(
    tagged_text,
    src_lang="en",
    tgt_lang="kor_Hang",
    do_sample=False,
    num_beams=4,
    max_length=1024  # 필수!
)[0]["translation_text"]


Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (10506 > 1024). Running this sequence through the model will result in indexing errors
Your input_length: 10506 is bigger than 0.9 * max_length: 1024. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [79]:
import re

# 태그별로 추출 (예: <SEG0> ... <SEG1> ...)
matches = re.split(r"<SEG\d+>", translated_text)
# 앞부분이 공백이거나 ""일 수 있으므로 제거
translated_segments = [seg.strip() for seg in matches if seg.strip()]

# 원래 순서 유지
for i, (orig, trans) in enumerate(zip(segments, translated_segments)):
    print(f"[{i}] {orig[:30]} → {trans[:30]}")


[0] aapl-20230701 → <SEGSEGSEGSEGSEGSEGSEGSEGSEGSE


In [ ]:
with open("data/translations/AAPL_2023_Q3_translated_list.txt", "w", encoding="utf-8") as f:
    for seg in translated_segments:
        f.write(seg + "\n\n␟\n\n")


몰라 ==-----=====


In [ ]:
from tqdm import tqdm

In [39]:
translations = []
for seg in tqdm(segments[:10]):
    if seg.strip():
        trans_result = translator(seg, src_lang="en", tgt_lang="kor_Hang", 
                    num_beams=5,             
                    penalty_alpha=None)     
        result = trans_result[0]['translation_text']
        
        
    else:
        result = ""
    translations.append(result)
    
dict(zip(segments[:10], translations))

 10%|█         | 1/10 [00:01<00:16,  1.82s/it]

[{'translation_text': 'aapl-20230701에 대한'}]


 20%|██        | 2/10 [00:04<00:17,  2.22s/it]

[{'translation_text': '미국에서는 물론이고, 미국에서는 물론이고, 미국에서도 마찬가지입니다.'}]


 30%|███       | 3/10 [00:07<00:20,  2.87s/it]

...

[{'translation_text': '(Mark One에서)'}]


 70%|███████   | 7/10 [00:16<00:07,  2.56s/it]

[{'translation_text': 'SECURITIES EXCHANGE ACT OF 1934 제13조 또는 15조의 4차 보고서를 제출합니다.'}]


 70%|███████   | 7/10 [00:22<00:09,  3.26s/it]


KeyboardInterrupt: 

{'aapl-20230701': 'aapl-20230701에 대한 설명',
 'UNITED STATES': '미국에서는 미군이',
 'SECURITIES AND EXCHANGE COMMISSION': '증권 및 교환위원회',
 'Washington, D.C. 20549': '워싱턴 D.C. 20549에 대한',
 'FORM 10-Q': 'FORM 10-Q의 형태는',
 '(Mark One)': '(Mark One) (Mark One)',
 'QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934': '13조 또는 15조에 따른 분기보고 (주) (d) 1934년 미국 증권거래법',
 'or': '또는 오렌지',
 'TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934': '13조 또는 15조에 의한 전환 보고서를 제출 (D) 1934년 미국 보안 거래소 법의',
 'For the transition period from': '이 기간 동안은 전환 기간이'}

In [38]:
translator("UNITED STATES", src_lang="en", tgt_lang="kor_Hang", 
                   do_sample=False,        # 샘플링 비활성화 → 핵심

                    num_beams=1,   
                    top_k = 1,
                    top_p = 1 ,      
                    penalty_alpha=None)[0]['translation_text']  

The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'미국에서는 미군이'

In [29]:
translations

['aapl-20230701에 대한 설명',
 '미국에서는 미군이',
 '증권 및 교환위원회',
 '워싱턴 D.C. 20549에 대한',
 'FORM 10-Q의 형태는',
 '(Mark One) (Mark One)',
 '13조 또는 15조에 따른 분기보고 (주) (d) 1934년 미국 증권거래법',
 '또는 오렌지',
 '13조 또는 15조에 의한 전환 보고서를 제출 (D) 1934년 미국 보안 거래소 법의',
 '이 기간 동안은 전환 기간이']

In [ ]:

# HTML 파일 불러오기
# load original segments
with open("data/segments/AAPL_2023_Q3_joined.txt", "r", encoding="utf-8") as f:
    segments = f.read().split("\n\n␟\n\n")

translations = []
for seg in segments:
    if seg.strip():
        result = translator(seg, src_lang="en", tgt_lang="kor_Hang",num_beams=2, penalty_alpha=0.6, top_k=4,)[0]['translation_text']
    else:
        result = ""
    translations.append(result)

# save to file
with open("data/translations/AAPL_2023_Q3_translated.txt", "w", encoding="utf-8") as f:
    f.write("\n\n␟\n\n".join(translations))


In [ ]:

# 번역 실행
# korean_text = translator(en, src_lang="en", tgt_lang="kor_Hang", num_beams=2, penalty_alpha=0.6, top_k=4)

In [ ]:

# 결과 출력
# print(korean_text[0]['translation_text'])

In [ ]:
# # english_text = "Hello, world! My name is AI."
# korean_text = translator(english_text, src_lang="en", tgt_lang="kor_Hang")

In [ ]:
# en = """amd-20231230

# ␟

# UNITED STATES SECURITIES AND EXCHANGE COMMISSION

# ␟

# Washington, D.C. 20549

# ␟

# FORM 10-K

# ␟

# (Mark One)

# ␟

# For the fiscal year ended

# ␟

# OR

# ␟

# For the transition period from

# ␟

# to

# ␟

# Commission File Number

# ␟

# ADVANCED MICRO DEVICES, INC.

# ␟

# (Exact name of registrant as specified in its charter)

# ␟

# Delaware

# ␟

# (State or other jurisdiction of incorporation or organization)

# ␟

# (I.R.S. Employer  Identification No.)

# ␟

# 2485 Augustine Drive



# """

In [ ]:
# korean_text = translator(en, src_lang="en", tgt_lang="kor_Hang")
# korean_text

In [ ]:
# korean_text = translator(en, src_lang="en", tgt_lang="kor_Hang", num_beams=2, penalty_alpha=0.6, top_k=4, )
# # korean_text